In [39]:
import dataflows as DF

In [40]:
SOURCES = [
    dict(
        year=2018, 
        filename='ספר שירותים 2018 - סופי.xlsx',
        sheets=[
            dict(
                office='משרד הבריאות',
                options=dict(
                    sheet='בריאות',
                    headers=2
                )
            ),
            dict(
                office='משרד הרווחה',
                options=dict(
                    sheet='רווחה',
                    headers=2
                )
            ),           
            dict(
                office='משרד החינוך',
                options=dict(
                    sheet='חינוך',
                    headers=2
                )
            ),
            dict(
                office='משרד העליה והקליטה',
                options=dict(
                    sheet='עליה וקליטה',
                    headers=2
                )
            ),
        ]
    ),

    dict(
        year=2017, 
        filename='ספר שירותים חברתיים 2017 לעיצוב.xlsx',
        sheets=[
            dict(
                office='משרד הבריאות',
                options=dict(
                    sheet='בריאות 2017',
                    headers=1
                )
            ),
            dict(
                office='משרד הרווחה',
                options=dict(
                    sheet='רווחה 2017 ',
                    headers=1
                )
            ),           
            dict(
                office='משרד החינוך',
                options=dict(
                    sheet='חינוך 2017',
                    headers=1
                )
            ),
            dict(
                office='משרד העליה והקליטה',
                options=dict(
                    sheet='עלייה וקליטה 2017',
                    headers=1
                )
            ),
        ]
    ),


    dict(
        year=2016, 
        filename='מיפוי שירותים חברתים בריאות 2016.xlsx',
        sheets=[
            dict(
                office='משרד הבריאות',
                options=dict(
                    sheet='גיליון1',
                    headers=2
                )
            ),
        ]
    ),
    
    dict(
        year=2016, 
        filename='מיפוי שירותים חברתיים רווחה 2016.xlsx',
        sheets=[
            dict(
                office='משרד הרווחה',
                options=dict(
                    sheet='רווחה',
                    headers=1
                )
            ),           
        ]
    ),

    dict(
        year=2016, 
        filename='מיפוי שירותים חברתיים חינוך 2016.xlsx',
        sheets=[
            dict(
                office='משרד החינוך',
                options=dict(
                    sheet='מטויב',
                    headers=2
                )
            ),
        ]
    ),

    dict(
        year=2016, 
        filename='מיפוי שירותים חברתיים קליטה 2016.xlsx',
        sheets=[
            dict(
                office='משרד העליה והקליטה',
                options=dict(
                    sheet='קליטה פתוח (2)',
                    headers=1
                )
            ),
        ]
    ),
]

In [41]:
URL_PATTERN = 'https://docs.google.com/spreadsheets/d/{id}/edit#gid={gid}'

In [42]:
loads = []
i = 0
for source in SOURCES:
    for sheet in source['sheets']:
        i += 1
        resource_name = 'res_{}'.format(i)
        url = source['filename']
        loads.append((
            resource_name,
            DF.Flow(
                DF.load(url, name=resource_name, **sheet.get('options', {})),
                DF.add_field('year', 'integer', source['year']),
                DF.add_field('publisher_name', 'string', sheet['office']),
            )
        ))

In [43]:
FIELD_MAPPING = dict(
    year=[],
    publisher_name=[],
    unit=['מינהל/ חטיבה', 'מינהל/ אגף', 'שם מינהל האגף', 'מנהל / אגף', 'מינהל'],
    subunit=['אגף', 'אגף/ מחלקה', 'שם האגף / מחלקה', 'אגף / מחלקה'],
    subsubunit=['מחלקה'],
    activity_name=['שם השירות', 'שם השירות החברתי', 'שם השירות  חברתי', 'שירות חברתי'],
    activity_description=['תיאור השירות'],
    allocated_budget=['תקציב (אלש"ח)', 'תקציב 2018 אלפי ₪', 'תקציב 2017 אלפי ₪', 'היקף התקשרות שנתי (אש"ח)', 'תקציב (מלש"ח)', 'תקציב', 'תקציב 2016 אלפי ₪'],
    num_beneficiaries=['מספר מקבלי שירות', 'מספר מקבלי השרות', 'מספר מקבלי  שירות'],
)

In [44]:
def multiply_budget(row):
    if row['allocated_budget']:
        row['allocated_budget'] = int(row['allocated_budget']) * 1000

def fill_org_hierarchy(rows):
    prev = {}
    for row in rows:
        unit = row.get('unit') or ''
        activity_name = row['activity_name']
        if 'סה״כ' in unit or 'סה"כ' in unit or 'סה"כ תקציב' in activity_name:
            continue
        for k, v in prev.items():
            if not row.get(k):
                row[k] = v
        yield row
        prev = dict(
            (k, v)
            for k, v in row.items()
            if k.endswith('unit')
        )

In [45]:
for resource_name, load in loads:
    DF.Flow(
        load,
        # DF.printer(tablefmt='html'),
        DF.concatenate(FIELD_MAPPING, dict(name=resource_name, path=resource_name+'.csv')),
        DF.set_type('activity_name', constraints=dict(required=True), on_error=DF.schema_validator.drop),
        DF.set_type('allocated_budget', type='number', groupChar=',', bareNumber=False),
        DF.set_type('num_beneficiaries', type='number', groupChar=',', bareNumber=False, on_error=DF.schema_validator.ignore),
        DF.set_type('num_beneficiaries', type='any'),
        multiply_budget,
        fill_org_hierarchy,
        # DF.printer(tablefmt='html'),
        DF.dump_to_path('tmp/' + resource_name),
    ).process()

In [46]:
yearly_fields = ['year', 'unit', 'subunit', 'subsubunit', 'allocated_budget', 'num_beneficiaries']
DF.Flow(
    *[
        DF.load('tmp/' + resource_name + '/datapackage.json')
        for resource_name, _ in loads
    ],
    DF.concatenate(FIELD_MAPPING, dict(name='social_services', path='social_services.csv')),
    DF.add_field('history', 'object', lambda r: dict(
        (k, r[k]) for k in yearly_fields   
    )),
    DF.join_with_self('social_services', ['publisher_name', 'activity_name'], dict(
        publisher_name=None,
        activity_name=None,
        activity_description=dict(aggregate='set'),
        min_year=dict(name='year', aggregate='min'),
        max_year=dict(name='year', aggregate='max'),
        history=dict(aggregate='array'),
    )),
    DF.printer(tablefmt='html'),
).process()

#,activity_name (any),activity_description (array),publisher_name (string),history (array),max_year (integer),min_year (integer)
1,'ביצוע בדיקת זכאות לנכות על פי חוק השיקום,[],משרד הבריאות,"[{'allocated_budget': Decimal('650000'), 'num_beneficiaries': 'מאות', 'subsubunit': None, 'subunit': ...",2017,2016
2,"אש""ל- מיזם משותף חברתי בריאותי לקשישים",[],משרד הבריאות,"[{'allocated_budget': Decimal('4000000'), 'num_beneficiaries': '24096', 'subsubunit': None, 'subunit ...",2017,2016
3,אשפוז סיעודי,[],משרד הבריאות,"[{'allocated_budget': Decimal('2140000000'), 'num_beneficiaries': '13600', 'subsubunit': None, 'subu ...",2017,2016
4,"אשפוזיות גמילה מסמים, אלכוהול וחומרים פסיכואקטיביים","['אשפוזיות גמילה ציבוריות בפריסה ארצית, בעלות רישיון כמוסד רפואי או משולב, לטיפול גמילה רפואי ופסיכו ...",משרד הבריאות,"[{'allocated_budget': Decimal('31025000'), 'num_beneficiaries': '2500', 'subsubunit': 'טיפול בהתמכרו ...",2018,2018
5,אשפוזיות- שירותי גמילה,[],משרד הבריאות,"[{'allocated_budget': Decimal('20000000'), 'num_beneficiaries': '2082', 'subsubunit': None, 'subunit ...",2017,2016
6,בדיקות גנטיות,"['בדיקות סקר גנטיות- לאיתור נשאים למחלות גנטיות, בדיקות אבחנתית בהריון ( סיסי שליה/מי שפיר) לאיתור ש ...",משרד הבריאות,"[{'allocated_budget': Decimal('100000'), 'num_beneficiaries': '76000', 'subsubunit': 'גנטיקה קהילתית ...",2018,2016
7,ביצוע סקרי הערכת מסגרות שיקום,[],משרד הבריאות,"[{'allocated_budget': Decimal('1500000'), 'num_beneficiaries': '1000', 'subsubunit': None, 'subunit' ...",2017,2016
8,בריאות השן ברשויות,[],משרד הבריאות,"[{'allocated_budget': Decimal('30000000'), 'num_beneficiaries': 'רשויות מקומיות', 'subsubunit': None ...",2017,2016
9,בריאות השן לתלמיד ברשויות מקומיות,"['שירות מונע בגני ילדים וסתי ספר הכולל בדיקת רופא שיניים, חינוך לבריאות והדרכה לשמירה על גהות הפה, ח ...",משרד הבריאות,"[{'allocated_budget': Decimal('40000000'), 'num_beneficiaries': '247', 'subsubunit': 'אגף לבריאות הש ...",2018,2018
10,בריאות התלמיד,[],משרד הבריאות,"[{'allocated_budget': Decimal('100000000'), 'num_beneficiaries': '1100000', 'subsubunit': None, 'sub ...",2017,2016


(<datapackage.package.Package at 0x11f520f90>, {})